# Individual Assignment 3 - Querying data
In this assignment, students are going to focus on getting data out of a database, using a few different methods

## Goal
The goal of this assignment is to understand how to get data out of a database using SQL queries directly, pandas and also directly into our objects using SQLAlchemy 

## Deliverables

## Background

## Instructions

# Part 1 - Querying data using SQL commands
Since you already know quite a bit about querying a SQL database, this is the easiest way to get data from a database.

The basic steps are 
1. *Connect to the database*. This may require some setup work depending on database type (SQLite, MS SQL, Teradata, etc).  Often times an intermediate piece of software called a database driver is required. This piece of software knows how to make a connection to the database server and send your SQL commands to the database and return the results to you. For SQLite, the driver is built-into Python and so there is no extra software to setup other than the standard Python libraries.
2. *Create a cursor* that can be used to send commands to the server. A database cursor is a pointer that moves around the database and points to the data you want to retrieve from the database.
3. *Execute a command*. Once the setup has been completed, we can finally execute a command. A command is the SQL statement you want to be sent to the database.  If the command returns data (as in a SELECT statement), then the cursor will point to the data that is to be returned (but keep in mind, only 1 row of data at a time).

The next cell shows the basics of creating a connection, establishing a cursor and executing a command to the database.

In [ ]:
import sqlite3

# Set the name of the database file to be used for this exercise
db_file = 'laptopsales.db'

# Make a connection to the database
cnn = sqlite3.connect(db_file)

# Next make a cursor that can be used to run a query on the new connection
cur = cnn.cursor()

# See what tables are available in the database
tables = cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(tables.fetchall())


# get the first 10 records from the sales table
result = cur.execute('SELECT * from sales LIMIT 10')

# Get the names of the columns from a sqlite database
names = list(map(lambda x: x[0], cur.description))
print(names)

# Print the first 10 records
for row in result:
    print(row)  

# It's good practice to ensure that we close our connection to the database when we are done using it
cnn.close()


Notice that each row of the query returns the result in a tuple.  A tuple similar to a list - it's a collection of values.  If we want to access a particular item in the results - we have to do so with an index.  Let's start by getting some data then we'll 

In [ ]:
# We have to reopen the database and create a new cursor to run queries on it
cnn = sqlite3.connect(db_file)
cur = cnn.cursor()

# We can also use the fetchall() method to get all the results at once
results = cur.execute('SELECT * from sales LIMIT 10').fetchall()
print(results)
    
# We can also use the fetchmany() method to get a specific number of results
results = cur.execute('SELECT * from sales LIMIT 10').fetchmany(5)
print(results)

In [ ]:
# Now lets get just a single result and print out the SalesID, and the Sale Date.
results = cur.execute('SELECT * from sales LIMIT 10').fetchone()
print(results)
# Print the SalesID and Sale Date.  The SalesID is the first column, and the Sale Date is the 17th column
print(f'SalesID: {results[0]}, Sale Date: {results[16]}')

## YOUR TURN - Queries
Now that we know which columns are available to us we can query the database let's run a few queries.
Answer the questions in the following cells.  Ensure that your code and the result are available before saving to PDF.

As a reference, here are the column names:
```
sale_id
Configuration
Customer Postcode
Store Postcode
Retail Price
Screen Size (Inches)
Battery Life (Hours)
RAM (GB)
Processor Speeds (GHz)
Integrated Wireless?
HD Size (GB)
Bundled Applications?
customer X
customer Y
store X
store Y
sale_date
```

In [ ]:
# The following steps are provided as a reminder of how to execute the query

# Open the database connection
# Create the cursor for which to execute the query
# Execute the query
# Print the results
#  HINT: Since we are only looking for a single value - fetchone() is a good method to use
# Close the connection

# Q1: What is the total revenue from sales in the sales table? (the answer should be 144373314.0)

In [ ]:
# Q2: Sum the sales by Store Postcode include the postcode in the results
# HINT: Now we are looking for multiple values - fetchall() is a good method to use

### Using the data from the queries
In all the examples the data comes back from the query as a tuple.  This means that we are getting all the values in a sort of collection of variables.  We can access the data in tuple just like it were a list.

Now, create a class that has a property for each of the columns in the database.
Query the database to get the first 20 items in the database, create an object from each one and put it in a list
Then loop through the list and print the Sale Id, Configuration and Sale date for all 20 items in list


In [ ]:
# Define the laptop sale class
# This class will be used to store the results of the query

# # Open the database connection
# Create the cursor for which to execute the query
# Execute the query

# Create an empty list to store the results
# Loop through the results and create a new LaptopSale object for each row
# Add the new LaptopSale object to the list

# Close the connection

# Loop through the list of objects
# Print the object's Sale Id, Configuration and Sale date

# Part 2 - Using pandas to connect to the database
If we are just looking to grab some data from a database and use it for analytics especially, then the [pandas](https://pandas.pydata.org/) library is a great choice.

To get data from SQL using `pandas` all we need to do is create the connection and execute the query


In [ ]:
# Create connection
import sqlite3
import pandas as pd

# Set the name of the database file to be used for this exercise
db_file = 'laptopsales.db'

# Make a connection to the database
cnn = sqlite3.connect(db_file)

data = pd.read_sql_query("SELECT * FROM sales LIMIT 10", cnn)
data

Now that our data is in a dataframe we can do all the things we know how to do with pandas already - like aggregate, filter, group and slice

In [ ]:
# Calculate the average battery life
mean_battery_life = data['Battery Life (Hours)'].mean()
print(f'Average Battery Life: {mean_battery_life} hours')

# Get all sales where the laptop has integrated wireless
wireless_sales = data[data['Integrated Wireless?'] == 'Yes']
print(f'wireless sales count: {len(wireless_sales)}')

# Price difference between items with and without integrated wireless
wireless = data[data['Integrated Wireless?'] == 'Yes']['Retail Price'].mean()
no_wireless = data[data['Integrated Wireless?'] == 'No']['Retail Price'].mean()
print(f'Price difference between items with and without integrated wireless: {wireless - no_wireless}')


## YOUR TURN - Using pandas to query data source
Now that you have seen how to connect to a database using pandas, we are going to move to a new database with a few more tables.  You can read about the chinook example [here](https://www.sqlitetutorial.net/sqlite-sample-database/).

![chinook database](images/sqlite-sample-database-color.jpg)

Complete the following queries and show your results

In [ ]:
# Create connection
# Write a query that will return the artist name, album title, and genre name into a dataframe
# Close the connection
# Show the dataframe

### Dynamic Query
For this next exercise you need to ask the user for a band name and then use this name to get all the albums for that band into a dataframe.

***HINTS:***
1. Consider building the query with string manipulation
2. It might be difficult to get the band name spelled exactly right, so use the SQL operator `LIKE` to help with getting close
3. To get user input you will use the `input()` function
4. Review [this lesson](https://www.py4analytics.info/book/lessons/250-Database_Connections.html#dynamic-queries) on dynamic queries if you need some additional help

In [ ]:
artist_name = input("What band are you interested in?  Put a partial name if you don't know the full name: ")
# Your code for the dynamic query goes here